In [1]:
#Import packages
import sqlite3
import pandas as pd

In [2]:
# Connect to the SQLite database file
conn = sqlite3.connect('sqlite_db_pythonsqlite.db')

In [3]:
#Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue.
#Remember that there's a different cost for guests and members!
q10_query = '''SELECT f.name AS facility_name,
        SUM(CASE
            WHEN b.memid != 0 THEN f.membercost * b.slots
            ELSE f.guestcost * b.slots END) AS total_revenue
FROM Facilities AS f
INNER JOIN Bookings AS b
ON f.facid = b.facid
GROUP BY f.name
HAVING total_revenue < 1000
ORDER BY total_revenue;'''
q10_df = pd.read_sql_query(q10_query, conn)
q10_df

,facility_name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


In [4]:
#Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order.
q11_query = '''SELECT
            (m.surname || ', ' || m.firstname) AS member_lastname_firstname,
            CASE WHEN m.recommendedby IS NULL THEN 'None'
            ELSE (r.surname || ', ' || r.firstname) END AS referrer_lastname_firstname
FROM Members AS m
LEFT JOIN Members AS r
ON m.recommendedby = r.memid
WHERE member_lastname_firstname NOT LIKE '%GUEST%'
ORDER BY member_lastname_firstname;'''
q11_df = pd.read_sql_query(q11_query, conn)
q11_df

,member_lastname_firstname,referrer_lastname_firstname
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Farrell, David",None
9,"Farrell, Jemima",None


In [5]:
#Q12: Find the facilities with their usage by member, but not guests.
q12_query = '''SELECT f.name AS facility,
                (m.surname || ', ' || m.firstname) AS member,
                COUNT(*) AS times_used
FROM Bookings AS b
INNER JOIN Facilities AS f
ON b.facid = f.facid
INNER JOIN Members as m
ON b.memid = m.memid
WHERE b.memid <> 0
GROUP BY facility, member
ORDER BY facility'''
q12_df = pd.read_sql_query(q12_query, conn)
q12_df

,facility,member,times_used
0,Badminton Court,"Bader, Florence",9
1,Badminton Court,"Baker, Anne",10
2,Badminton Court,"Baker, Timothy",7
3,Badminton Court,"Boothe, Tim",12
4,Badminton Court,"Butters, Gerald",20
...,...,...,...
197,Tennis Court 2,"Smith, Darren",19
198,Tennis Court 2,"Smith, Jack",1
199,Tennis Court 2,"Smith, Tracy",2
200,Tennis Court 2,"Stibbons, Ponder",31


In [6]:
#Q13: Find the facilities usage by month, but not guests.
q13_query = '''SELECT f.name AS facility,
                strftime('%m', starttime) AS month,
                COUNT(*) AS times_used
FROM Bookings AS b
INNER JOIN Facilities AS f
ON b.facid = f.facid
WHERE b.memid <> 0
GROUP BY facility, month
ORDER BY facility'''
q13_df = pd.read_sql_query(q13_query, conn)
q13_df

,facility,month,times_used
0,Badminton Court,07,51
1,Badminton Court,08,132
2,Badminton Court,09,161
3,Massage Room 1,07,77
4,Massage Room 1,08,153
5,Massage Room 1,09,191
6,Massage Room 2,07,4
7,Massage Room 2,08,9
8,Massage Room 2,09,14
9,Pool Table,07,103


In [7]:
# Close the connection
conn.close()